# Анализ продаж автомобилей в Казахстане в 2019 году

## Загрузка данных и просмотр общей информации

In [2]:
import pandas as pd
import re

In [3]:
df = pd.read_csv('autokz2019.csv',decimal =',', sep=';', skipinitialspace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39966 entries, 0 to 39965
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Год                             39966 non-null  int64  
 1   Месяц                           39966 non-null  object 
 2   Компания                        39966 non-null  object 
 3   Бренд                           39966 non-null  object 
 4   Модель                          39966 non-null  object 
 5   Модификация                     36375 non-null  object 
 6   Год выпуска                     39465 non-null  object 
 7   Страна-производитель            39966 non-null  object 
 8   Вид топлива                     36826 non-null  object 
 9   Объём двиг, л,                  35708 non-null  object 
 10  Коробка передач                 36711 non-null  object 
 11  Тип привода                     35677 non-null  object 
 12  Сегмент                         

удаляем столбцы: наименование дилерского центра, форма расчета, сегмент, сегментация eng, локализация производства, тип клиента и модификация (они либо бесполезны, либо много пропусков)

In [5]:
df = df.drop(df.columns[[5, 12, 14, 15, 16, 23, 24]], axis=1)

In [6]:
df.head()

,Год,Месяц,Компания,Бренд,Модель,Год выпуска,Страна-производитель,Вид топлива,"Объём двиг, л,",Коробка передач,Тип привода,Регион,Количество,"Цена, USD","Продажа, USD",Область,Сегментация 2013,Класс 2013
0,2019,Май,Mercur Auto,Audi,A3,2018,Германия,Бензин,"1,4",S-tronic,передний,Алматы,1.0,28115.00,28115.00,г.Алматы,Легковые автомобили,C класс
1,2019,Август,Mercur Auto,Audi,A3,2018,Германия,Бензин,"1,4",S-tronic,передний,Алматы,1.0,32246.99,32246.99,г.Алматы,Легковые автомобили,C класс
2,2019,Апрель,Mercur Auto,Audi,A4,2018,Германия,Бензин,"1,4",S-Tronic,FWD,Алматы,1.0,32000.00,32000.00,г.Алматы,Легковые автомобили,D класс
3,2019,Июль,Mercur Auto,Audi,A4,2018,Германия,Бензин,"1,4",S-tronic,передний,Алматы,1.0,31929.00,31929.00,г.Алматы,Легковые автомобили,D класс
4,2019,Июль,Mercur Auto,Audi,A4,2018,Германия,Бензин,"1,4",S-tronic,передний,Алматы,1.0,31929.00,31929.00,г.Алматы,Легковые автомобили,D класс


Изменим наименования столбцов

In [7]:
df=df.rename(columns={
    'Год': 'year', 'Месяц': 'month', 'Компания': 'company', 'Бренд': 'brand', 'Модель': 'model', 'Год выпуска': 'year_of_release', \
    'Страна-производитель': 'country_of_manufacture', 'Вид топлива': 'fuel', 'Объём двиг, л,': 'engine_volume', 'Коробка передач': 'transmission_box', \
    'Тип привода': 'type_of_drive', 'Регион': 'region', 'Количество': 'quantity', 'Цена, USD': 'price', 'Продажа, USD': 'sale', \
    'Область': 'oblact', 'Сегментация 2013': 'segment', 'Класс 2013': 'class'})

In [8]:
df.head()

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
0,2019,Май,Mercur Auto,Audi,A3,2018,Германия,Бензин,"1,4",S-tronic,передний,Алматы,1.0,28115.00,28115.00,г.Алматы,Легковые автомобили,C класс
1,2019,Август,Mercur Auto,Audi,A3,2018,Германия,Бензин,"1,4",S-tronic,передний,Алматы,1.0,32246.99,32246.99,г.Алматы,Легковые автомобили,C класс
2,2019,Апрель,Mercur Auto,Audi,A4,2018,Германия,Бензин,"1,4",S-Tronic,FWD,Алматы,1.0,32000.00,32000.00,г.Алматы,Легковые автомобили,D класс
3,2019,Июль,Mercur Auto,Audi,A4,2018,Германия,Бензин,"1,4",S-tronic,передний,Алматы,1.0,31929.00,31929.00,г.Алматы,Легковые автомобили,D класс
4,2019,Июль,Mercur Auto,Audi,A4,2018,Германия,Бензин,"1,4",S-tronic,передний,Алматы,1.0,31929.00,31929.00,г.Алматы,Легковые автомобили,D класс


Проверим датасет на пропуски 

In [9]:
report = df.isna().sum().to_frame()
report = report.rename(columns = {0: 'missing_values'})
report['% of total'] = ((report['missing_values'] / df.shape[0])*100).round(2)
report.sort_values(by = 'missing_values', ascending = False)

,missing_values,% of total
type_of_drive,4289,10.73
engine_volume,4258,10.65
transmission_box,3255,8.14
fuel,3140,7.86
year_of_release,501,1.25
quantity,6,0.02
region,0,0.00
segment,0,0.00
oblact,0,0.00
sale,0,0.00


In [10]:
df[df['year_of_release'].isna()]

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
65,2019,Июнь,Автоцентр-Бавария,BMW,7 серия,NaN,Германия,Бензин,3,АКПП,Полный,Усть-Каменогорск,1.0,93520.28222,93520.28222,Восточно-Казахстанская область,Легковые автомобили,F класс
9599,2019,Май,Hyundai Com Trans Kazakhstan,Hyundai Truck & Bus,H350,NaN,Республика Казахстан,Дизель,NaN,NaN,2WD,Алматы,1.0,33833.35184,33833.35184,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
9600,2019,Май,Hyundai Com Trans Kazakhstan,Hyundai Truck & Bus,H350,NaN,Республика Казахстан,Дизель,NaN,NaN,2WD,Алматы,1.0,33833.35184,33833.35184,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
9601,2019,Май,Hyundai Com Trans Kazakhstan,Hyundai Truck & Bus,H350,NaN,Республика Казахстан,Дизель,NaN,NaN,2WD,Алматы,1.0,33833.35184,33833.35184,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
9602,2019,Май,Hyundai Com Trans Kazakhstan,Hyundai Truck & Bus,H350,NaN,Республика Казахстан,Дизель,NaN,NaN,2WD,Алматы,1.0,33833.35184,33833.35184,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39542,2019,Сентябрь,Volkswagen Group Rus,Volkswagen,Teramont,NaN,Российская Федерация,NaN,NaN,NaN,NaN,Костанай,1.0,49655.99915,49655.99915,Костанайская область,Внедорожники,Полноразмерные SUV
39554,2019,Июнь,Hyundai Com Trans Kazakhstan,Hyundai Truck & Bus,EX8,NaN,Республика Казахстан,NaN,NaN,NaN,NaN,Алматы,1.0,34590.76936,34590.76936,г.Алматы,Коммерческие автомобили,Малотоннажные грузовики
39555,2019,Июнь,Hyundai Com Trans Kazakhstan,Hyundai Truck & Bus,EX8,NaN,Республика Казахстан,NaN,NaN,NaN,NaN,Алматы,1.0,34590.76936,34590.76936,г.Алматы,Коммерческие автомобили,Малотоннажные грузовики
39556,2019,Июнь,Hyundai Com Trans Kazakhstan,Hyundai Truck & Bus,EX8,NaN,Республика Казахстан,NaN,NaN,NaN,NaN,Алматы,1.0,34590.76936,34590.76936,г.Алматы,Коммерческие автомобили,Малотоннажные грузовики


Так как количество пропусков в столбцах 'quantity' и'year_of_release' невелико, удалим их из датасета

In [11]:
df=df.dropna(axis=0, subset=['quantity','year_of_release'])

## Обработка данных в столбцах

### Год выпуска автомобиля

In [12]:
# посмотрим уникальные значения
df['year_of_release'].unique()

array(['2018', '2017', '2019', '2016', '2014', '2013', '2011', '2\xa0018',
       '2\xa0019'], dtype=object)

In [13]:
# скомпилируем регулярные выражения, чтобы ускорить их применение. Здесь '\D' удаляет все символы, не являющиеся цифрами, в столбце 'year_of_release'.

magic_wand = re.compile(r'\D') 
df['year_of_release'] = df['year_of_release'].map(lambda x: magic_wand.sub('', x))

In [14]:
df_release=df['year_of_release'].value_counts(dropna=False).to_frame(name='Количество')
df_release['Процент (%)'] = (df_release['Количество'] / len(df) * 100).round(2)

In [15]:
print("\nСтатистика по 'year_of_release':")
print(df_release.head(10))


Статистика по 'year_of_release':
      Количество  Процент (%)
2019       30023        76.09
2018        9078        23.01
2017         324         0.82
2016          30         0.08
2014           2         0.01
2013           1         0.00
2011           1         0.00


На авторынке Казахстана в 2019 году 76% проданных авто - 2019 года выпуска, 23% - 2018 года выпуска. 

### Наименования компаний-дилеров

In [16]:
# посмотрим уникальные значения
df['company'].unique()

array(['Mercur Auto', 'Автоцентр-Бавария', 'БИПЭК АВТО', 'Вираж',
       'Eurasia Motor Premium', 'Daewoo Bus Kazakhstan', 'Caspian Motors',
       'Каспиан Моторс', 'СемАЗ', 'Astana Motors', 'Astana Motors ',
       'Hyundai Com Trans Kazakhstan ', 'Nissan Manufacturing RUS',
       'СВС-ТРАНС', 'Allur Auto', 'Almaty Motors Premium', 'TERRA MOTORS',
       'ТК КАМАЗ', 'Toyota Motor Kazakhstan', 'Лифан Моторс Рус',
       'MAN Truck & Bus Kazakhstan', 'Автомир ГК', 'Autokapital',
       'Автокапитал', 'ММС Рус', 'MMC RUS', 'Mercur Autos',
       'УзАвто-Казахстан', 'Равон Моторс Казахстан',
       'Ravon Motors Kazakstan', 'Renault Россия', 'Scania Central Asia',
       'Subaru Kazakhstan', 'Volkswagen Group Rus', 'Автодом Motors KST',
       'Автомир-Центр', 'ТОО "Eurasia Motor Zhaik"',
       'Scandinavian Motors', 'Хино Моторс Казахстан', 'Hino Motors '],
      dtype=object)

In [17]:
# приводим всё к строке и удаляем лишние пробелы
df['company'] = df['company'].astype(str).str.strip()

По данным сайта 2gis.kz, наименования 'Mercur Autos' и 'Mercur Auto', 'Автомир ГК' и 'Автомир-Центр' относятся к одной и той же компании. Также дублируются наименования на русском и английском языках: 'Хино Моторс Казахстан' и 'Hino Motors', 'Каспиан Моторс' и 'Caspian Motors', 'Автокапитал' и 'Autokapital'.
Изменим значения столбца 'company'

In [18]:
df['company']= df['company'].replace({'Хино Моторс Казахстан':'Hino Motors', 'Каспиан Моторс':'Caspian Motors', 'Автокапитал':'Autokapital', 'Mercur Autos':'Mercur Auto', 'Автомир ГК':'Автомир-Центр'})

In [19]:
df_company=df['company'].value_counts(dropna=False).to_frame(name='Количество')
df_company['Процент (%)'] = (df_company['Количество'] / len(df) * 100).round(2)

In [20]:
print("\nСтатистика по 'company':")
print(df_company)


Статистика по 'company':
                              Количество  Процент (%)
Toyota Motor Kazakhstan            11951        30.29
Astana Motors                      10169        25.77
БИПЭК АВТО                          3618         9.17
Вираж                               2909         7.37
Allur Auto                          2305         5.84
Renault Россия                      1796         4.55
Nissan Manufacturing RUS            1149         2.91
Равон Моторс Казахстан               947         2.40
ТК КАМАЗ                             893         2.26
Mercur Auto                          643         1.63
Ravon Motors Kazakstan               559         1.42
Subaru Kazakhstan                    503         1.27
ММС Рус                              362         0.92
Лифан Моторс Рус                     233         0.59
Автоцентр-Бавария                    223         0.57
СВС-ТРАНС                            202         0.51
УзАвто-Казахстан                     142         0.36
Au

In [21]:
# посмотрим уникальные значения
df['brand'].unique()

array(['Audi', 'BMW', 'Chevrolet', 'Daewoo', 'Ford', 'Foton', 'GAZ',
       'Hyundai', 'Hyundai Truck & Bus', 'Infiniti', 'Isuzu', 'Iveco',
       'Jac', 'Jaguar', 'KAMAZ', 'Kia', 'Lada', 'Land Rover', 'Lexus',
       'Lifan', 'MAN', 'Mazda', 'Mercedes-Benz', 'Mercedes-Benz Trucks',
       'Mini', 'Mitsubishi', 'Nissan', 'Peugeot', 'Porsche', 'Ravon',
       'Renault', 'Scania', 'Skoda', 'Subaru', 'Toyota', 'UAZ',
       'Volkswagen', 'Volvo', 'Урал', 'Cadillac', 'Shacman', 'Hino',
       'Dong Feng', 'ANKAI', 'Nefaz', 'Yutong'], dtype=object)

In [22]:
df_brand=df['brand'].value_counts(dropna=False).to_frame(name='Количество')
df_brand['Процент (%)'] = (df_brand['Количество'] / len(df) * 100).round(2)

In [23]:
print("\nСтатистика по 'brand':")
print(df_brand)


Статистика по 'brand':
                      Количество  Процент (%)
Toyota                     10745        27.23
Hyundai                    10168        25.77
Lada                        1967         4.98
GAZ                         1821         4.61
Renault                     1796         4.55
Ravon                       1784         4.52
Jac                         1770         4.49
Lexus                       1206         3.06
Nissan                      1053         2.67
UAZ                          925         2.34
KAMAZ                        892         2.26
Kia                          831         2.11
Volkswagen                   682         1.73
Chevrolet                    635         1.61
Subaru                       504         1.28
Skoda                        378         0.96
Mitsubishi                   364         0.92
ANKAI                        300         0.76
Lifan                        233         0.59
BMW                          208         0.53
Isuzu     

In [24]:
df['model'].unique()

array(['A3', 'A4', 'A6', 'A7', 'A8', 'Q5', 'Q7', '3 серия', '5 серия',
       '6 серия', '7 серия', 'X1', 'X3', 'X4', 'X5', 'X6', 'X7', 'Z4',
       'Aveo', 'Captiva', 'Niva', 'Tahoe', 'Tracker', 'BS 106 A',
       'Fiesta', 'Mondeo', 'BJ3253DMPKB-AD', '2217', '2310', '2705',
       '2752', '3221', '3302', '3308', '3309', 'Next', 'Accent', 'Creta',
       'Elantra', 'Santa Fe', 'Sonata', 'TQ-1/H-1', 'County', 'H350',
       'HD 170', 'HD 65', 'HD 78', 'Universe', 'Tucson', 'Q50', 'QX50',
       'QX60', 'QX80', 'D-MAX', 'NMR', 'NPR', 'Daily', 'Daily A',
       'Daily V', 'Stralis', 'Tipper', 'S3', 'S5', 'F-Pace', 'F-Type',
       'XE', 'XF', 'XJ', '4308', '4311', '43118', '43253', '43502',
       '44108', '45142', '45143', '53215', '53504', '53605', '54115',
       '5490', '65111', '65115', '65116', '65117', '6520', '65206',
       '65225', '66052', '6606', 'Cerato', 'Optima', 'Picanto', 'Quoris',
       'Rio', 'Sorento', 'Soul', 'Sportage', '4x4', 'Granta', 'Kalina',
       'Largus', '

In [25]:
df['country_of_manufacture'].unique()

array(['Германия', 'США', 'Австрия', 'Республика Казахстан',
       'Российская Федерация', 'Корея', 'Япония', 'Таиланд', 'Китай',
       'UK', 'Узбекистан', 'Венгрия', 'Турция', 'Испания', 'Нидерланды',
       'Польша', 'Швеция', 'Белоруссия', 'Бельгия'], dtype=object)

кодируем страны в столбце 'страна производства' в alpha3 (RUS, KAZ, USA и т.д.)


In [26]:
df['country_of_manufacture'] = df['country_of_manufacture'].replace({
    'Германия': 'DEU',
    'США': 'USA',
    'Австрия': 'AUT', 
    'Республика Казахстан': 'KAZ',
    'Российская Федерация': 'RUS',
    'Корея': 'KOR',
    'Япония': 'JPN',
    'Таиланд': 'THA',
    'Китай': 'CHN',
    'UK': 'GBR',
    'Узбекистан': 'UZB',
    'Венгрия': 'HUN',
    'Турция': 'TUR',
    'Испания': 'ESP',
    'Нидерланды': 'NLD',
    'Польша': 'POL',
    'Швеция': 'SWE', 
    'Белоруссия': 'BLR', 
    'Бельгия': 'BEL'
})    

In [27]:
df_country=df['country_of_manufacture'].value_counts(dropna=False).to_frame(name='Количество')
df_country['Процент (%)'] = (df_country['Количество'] / len(df) * 100).round(2)

In [29]:
print("\nСтатистика по 'country_of_manufacture':")
print(df_country)


Статистика по 'country_of_manufacture':
     Количество  Процент (%)
KAZ       19038        48.25
RUS       12702        32.19
JPN        4591        11.63
TUR        1167         2.96
THA         959         2.43
USA         289         0.73
DEU         257         0.65
GBR         190         0.48
CHN          94         0.24
UZB          70         0.18
AUT          33         0.08
KOR          17         0.04
SWE          16         0.04
NLD          12         0.03
ESP           9         0.02
POL           6         0.02
BEL           6         0.02
HUN           2         0.01
BLR           1         0.00


### Обработка значений столбца 'тип топлива'

In [30]:
df['fuel'].unique()

array(['Бензин', 'Дизель', nan, 'дизель ', 'бензин ', 'гибрид', '2',
       '1,6', '0', 'Электро', 'Электричество'], dtype=object)

In [31]:
df[df['fuel']=='2']

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
23723,2019,Январь,Renault Россия,Renault,Duster,2019,RUS,2,MT,4WD,4WD,Уральск,1.0,12249.61125,12249.61125,Западно-Казахстанская область,Внедорожники,Субкомпактные SUV
23724,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,AT,4WD,4WD,Нур-Султан,1.0,12249.61125,12249.61125,г.Нур-Султан,Внедорожники,Субкомпактные SUV
23725,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,AT,4WD,4WD,Нур-Султан,1.0,12249.61125,12249.61125,г.Нур-Султан,Внедорожники,Субкомпактные SUV
23726,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,AT,4WD,4WD,Кызылорда,1.0,12249.61125,12249.61125,Кызылординская область,Внедорожники,Субкомпактные SUV
23727,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,MT,4WD,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23729,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,AT,4WD,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23730,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,AT,4WD,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23731,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,MT,4WD,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23732,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,MT,4WD,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23746,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,AT,4WD,4WD,Усть-Каменогорск,1.0,12249.61125,12249.61125,Восточно-Казахстанская область,Внедорожники,Субкомпактные SUV


In [32]:
df[df['fuel']=='1,6']

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
23728,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",MT,4WD,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23733,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",MT,4WD,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23734,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",AT,Передний,Передний,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23735,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",AT,Передний,Передний,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23736,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",AT,Передний,Передний,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23751,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",AT,Передний,Передний,Усть-Каменогорск,1.0,12249.61125,12249.61125,Восточно-Казахстанская область,Внедорожники,Субкомпактные SUV
23754,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",MT,4WD,4WD,Алматы,1.0,12249.61125,12249.61125,г.Алматы,Внедорожники,Субкомпактные SUV
23763,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",MT,4WD,4WD,Нур-Султан,1.0,12249.61125,12249.61125,г.Нур-Султан,Внедорожники,Субкомпактные SUV
23764,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",MT,4WD,4WD,Петропавловск,1.0,12249.61125,12249.61125,Северо-Казахстанская область,Внедорожники,Субкомпактные SUV
24997,2019,Январь,Renault Россия,Renault,Sandero,2018,RUS,"1,6",0,0,0,Атырау,1.0,12476.89637,12476.89637,Атырауская область,Легковые автомобили,B класс


In [37]:
df[df['fuel']=='0']

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
37121,2019,Январь,БИПЭК АВТО,UAZ,3741,2018,KAZ,0,"2,693",5 МТ,Полный,Талдыкорган,1.0,14200.0,14200.0,Алматинская область,Коммерческие автомобили,Малотоннажные грузовики
37122,2019,Январь,БИПЭК АВТО,UAZ,3741,2018,KAZ,0,"2,693",5 МТ,Полный,Актобе,3.0,14200.0,42600.0,Актюбинская область,Коммерческие автомобили,Малотоннажные грузовики
37123,2019,Январь,БИПЭК АВТО,UAZ,3741,2018,KAZ,0,"2,693",5 МТ,Полный,Актау,2.0,14200.0,28400.0,Мангистауская область,Коммерческие автомобили,Малотоннажные грузовики


Renault Duster 1.6 и 2.0 2018 и 2019 года выпуска - с бензиновым двигателем, UAZ 3741 - также работает на бензине

Кодируем тип топлива в (F - fuel, D - diesel, E - electro, HYB - hybrid) и меняем значения в столбце


In [41]:
df['fuel'] = df['fuel'].astype(str).str.strip()

In [42]:
df['fuel'] = df['fuel'].str.lower()

In [46]:
# 
df['fuel'] = df['fuel'].replace({'бензин': 'F', '0':'F','1,6':'F','2':'F','дизель': 'D', 'электро': 'E', 'электричество':'E', 'гибрид': 'HYB'})

In [47]:
df['fuel'].unique()

array(['F', 'D', 'nan', 'HYB', 'E'], dtype=object)

In [48]:
df[df['fuel']=='nan']

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
861,2019,Март,СемАЗ,Foton,BJ3253DMPKB-AD,2019,KAZ,nan,NaN,NaN,NaN,Алматы,1.0,48275.86207,48275.86207,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
862,2019,Июнь,СемАЗ,Foton,BJ3253DMPKB-AD,2018,KAZ,nan,NaN,NaN,NaN,Усть-Каменогорск,1.0,49735.91972,49735.91972,Восточно-Казахстанская область,Коммерческие автомобили,Крупнотоннажные грузовики
863,2019,Июнь,СемАЗ,Foton,BJ3253DMPKB-AD,2018,KAZ,nan,NaN,NaN,NaN,Кокшетау,1.0,48784.39304,48784.39304,Акмолинская область,Коммерческие автомобили,Крупнотоннажные грузовики
864,2019,Июль,СемАЗ,Foton,BJ3253DMPKB-AD,2018,KAZ,nan,NaN,NaN,NaN,Алматы,5.0,48725.42295,243627.11480,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
865,2019,Август,СемАЗ,Foton,BJ3253DMPKB-AD,2018,KAZ,nan,NaN,NaN,NaN,Алматы,2.0,31031.65229,62063.30457,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39943,2019,Сентябрь,Renault Россия,Renault,Arkana,2019,RUS,nan,NaN,NaN,NaN,Нур-Султан,1.0,21327.27747,21327.27747,г.Нур-Султан,Внедорожники,Компактные SUV
39944,2019,Сентябрь,Renault Россия,Renault,Arkana,2019,RUS,nan,NaN,NaN,NaN,Уральск,1.0,20889.17526,20889.17526,Западно-Казахстанская область,Внедорожники,Компактные SUV
39945,2019,Сентябрь,Renault Россия,Renault,Arkana,2019,RUS,nan,NaN,NaN,NaN,Атырау,1.0,20889.17526,20889.17526,Атырауская область,Внедорожники,Компактные SUV
39946,2019,Сентябрь,Renault Россия,Renault,Arkana,2019,RUS,nan,NaN,NaN,NaN,Уральск,1.0,20889.17526,20889.17526,Западно-Казахстанская область,Внедорожники,Компактные SUV


In [ ]:

# 4. Группируем редкие страны (<1%) в 'другие'
country_freq = df['country'].value_counts(normalize=True)
rare_countries = country_freq[country_freq < 0.01].index


In [ ]:
df['country'] = df['country'].replace(rare_countries, 'другие')


In [ ]:
# 5. Красивый формат
df['country'] = df['country'].str.title()

In [ ]:
comment_stats = df['country'].value_counts(dropna=False).to_frame(name='Количество')

In [ ]:
comment_stats['Процент (%)'] = (comment_stats['Количество'] / len(df) * 100).round(2)

In [ ]:
print("\nСтатистика по 'country':")
print(comment_stats.head(50))